In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
import torchtext.experimental
import torchtext.experimental.vectors
from torchtext.experimental.datasets.raw.text_classification import RawTextIterableDataset
from torchtext.experimental.datasets.text_classification import TextClassificationDataset
from torchtext.experimental.functional import sequential_transforms, vocab_func, totensor

import collections
import random
import time

In [2]:
seed = 1234

torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
raw_train_data, raw_test_data = torchtext.experimental.datasets.raw.IMDB()

In [4]:
def get_train_valid_split(raw_train_data, split_ratio = 0.7):

    raw_train_data = list(raw_train_data)
        
    random.shuffle(raw_train_data)
        
    n_train_examples = int(len(raw_train_data) * split_ratio)
        
    train_data = raw_train_data[:n_train_examples]
    valid_data = raw_train_data[n_train_examples:]
    
    train_data = RawTextIterableDataset(train_data)
    valid_data = RawTextIterableDataset(valid_data)
    
    return train_data, valid_data

In [5]:
raw_train_data, raw_valid_data = get_train_valid_split(raw_train_data)

In [6]:
class Tokenizer:
    def __init__(self, tokenize_fn = 'basic_english', lower = True, max_length = None):
        
        self.tokenize_fn = torchtext.data.utils.get_tokenizer(tokenize_fn)
        self.lower = lower
        self.max_length = max_length
        
    def tokenize(self, s):
        
        tokens = self.tokenize_fn(s)
        
        if self.lower:
            tokens = [token.lower() for token in tokens]
            
        if self.max_length is not None:
            tokens = tokens[:max_length]
            
        return tokens

In [7]:
max_length = 500

tokenizer = Tokenizer(max_length = max_length)

In [8]:
def build_vocab_from_data(raw_data, tokenizer, **vocab_kwargs):
    
    token_freqs = collections.Counter()
    
    for label, text in raw_data:
        tokens = tokenizer.tokenize(text)
        token_freqs.update(tokens)
                
    vocab = torchtext.vocab.Vocab(token_freqs, **vocab_kwargs)
    
    return vocab

In [9]:
max_size = 25_000

vocab = build_vocab_from_data(raw_train_data, tokenizer, max_size = max_size)

In [10]:
def process_raw_data(raw_data, tokenizer, vocab):
    
    raw_data = [(label, text) for (label, text) in raw_data]

    text_transform = sequential_transforms(tokenizer.tokenize,
                                           vocab_func(vocab),
                                           totensor(dtype=torch.long))
    
    label_transform = sequential_transforms(totensor(dtype=torch.long))

    transforms = (label_transform, text_transform)

    dataset = TextClassificationDataset(raw_data,
                                        vocab,
                                        transforms)
    
    return dataset

In [11]:
train_data = process_raw_data(raw_train_data, tokenizer, vocab)
valid_data = process_raw_data(raw_valid_data, tokenizer, vocab)
test_data = process_raw_data(raw_test_data, tokenizer, vocab)

In [12]:
class Collator:
    def __init__(self, pad_idx):
        
        self.pad_idx = pad_idx
        
    def collate(self, batch):
        
        labels, text = zip(*batch)
        
        labels = torch.LongTensor(labels)
        
        lengths = torch.LongTensor([len(x) for x in text])

        text = nn.utils.rnn.pad_sequence(text, padding_value = self.pad_idx)
        
        return labels, text, lengths

In [13]:
pad_token = '<pad>'
pad_idx = vocab[pad_token]
collator = Collator(pad_idx)

In [14]:
batch_size = 256

train_iterator = torch.utils.data.DataLoader(train_data, 
                                             batch_size, 
                                             shuffle = True, 
                                             collate_fn = collator.collate)

valid_iterator = torch.utils.data.DataLoader(valid_data, 
                                             batch_size, 
                                             shuffle = False, 
                                             collate_fn = collator.collate)

test_iterator = torch.utils.data.DataLoader(test_data, 
                                            batch_size, 
                                            shuffle = False, 
                                            collate_fn = collator.collate)

In [15]:
class BiLSTM(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, n_layers, dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.lstm = nn.LSTM(emb_dim, hid_dim, num_layers = n_layers, bidirectional = True, dropout = dropout)
        self.fc = nn.Linear(2 * hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, lengths):

        # text = [seq len, batch size]
        # lengths = [batch size]

        embedded = self.dropout(self.embedding(text))

        # embedded = [seq len, batch size, emb dim]

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths, enforce_sorted = False)

        packed_output, (hidden, cell) = self.lstm(packed_embedded)

        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output)

        # outputs = [seq_len, batch size, n directions * hid dim]
        # hidden = [n layers * n directions, batch size, hid dim]

        hidden_fwd = hidden[-2]
        hidden_bck = hidden[-1]

        # hidden_fwd/bck = [batch size, hid dim]

        hidden = torch.cat((hidden_fwd, hidden_bck), dim = 1)

        # hidden = [batch size, hid dim * 2]

        prediction = self.fc(self.dropout(hidden))

        # prediction = [batch size, output dim]

        return prediction

In [16]:
input_dim = len(vocab)
emb_dim = 100
hid_dim = 256
output_dim = 2
n_layers = 2
dropout = 0.5

model = BiLSTM(input_dim, emb_dim, hid_dim, output_dim, n_layers, dropout, pad_idx)

In [17]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [18]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,811,370 trainable parameters


In [19]:
glove = torchtext.experimental.vectors.GloVe(name = '6B',
                                             dim = emb_dim)

In [20]:
def get_pretrained_embedding(vectors, vocab, unk_token):
    
    unk_vector = vectors[unk_token]
    emb_dim = unk_vector.shape[-1]
    zero_vector = torch.zeros(emb_dim)

    pretrained_embedding = torch.zeros(len(vocab), emb_dim) 
    
    unk_tokens = []
    
    for idx, token in enumerate(vocab.itos):
        pretrained_vector = vectors[token]
        if torch.all(torch.eq(pretrained_vector, unk_vector)):
            unk_tokens.append(token)
            pretrained_embedding[idx] = zero_vector
        else:
            pretrained_embedding[idx] = pretrained_vector
        
    return pretrained_embedding, unk_tokens

In [21]:
unk_token = '<unk>'

pretrained_embedding, unk_tokens = get_pretrained_embedding(glove.vectors, vocab, unk_token)

In [22]:
model.embedding.weight.data.copy_(pretrained_embedding)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2925,  0.1087,  0.7920,  ..., -0.3641,  0.1822, -0.4104],
        [-0.7250,  0.7545,  0.1637,  ..., -0.0144, -0.1761,  0.3418],
        [ 1.1753,  0.0460, -0.3542,  ...,  0.4510,  0.0485, -0.4015]])

In [23]:
optimizer = optim.Adam(model.parameters())

In [24]:
criterion = nn.CrossEntropyLoss()

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
model = model.to(device)
criterion = criterion.to(device)

In [27]:
def calculate_accuracy(predictions, labels):
    top_predictions = predictions.argmax(1, keepdim = True)
    correct = top_predictions.eq(labels.view_as(top_predictions)).sum()
    accuracy = correct.float() / labels.shape[0]
    return accuracy

In [28]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, text, lengths in iterator:
        
        labels = labels.to(device)
        text = text.to(device)

        optimizer.zero_grad()
        
        predictions = model(text, lengths)
        
        loss = criterion(predictions, labels)
        
        acc = calculate_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for labels, text, lengths in iterator:

            labels = labels.to(device)
            text = text.to(device)
            
            predictions = model(text, lengths)
            
            loss = criterion(predictions, labels)
            
            acc = calculate_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
n_epochs = 10

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'bilstm-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 24s
	Train Loss: 0.656 | Train Acc: 60.38%
	 Val. Loss: 0.609 |  Val. Acc: 64.97%
Epoch: 02 | Epoch Time: 0m 24s
	Train Loss: 0.594 | Train Acc: 69.33%
	 Val. Loss: 0.578 |  Val. Acc: 71.96%
Epoch: 03 | Epoch Time: 0m 24s
	Train Loss: 0.565 | Train Acc: 70.59%
	 Val. Loss: 0.476 |  Val. Acc: 77.87%
Epoch: 04 | Epoch Time: 0m 24s
	Train Loss: 0.487 | Train Acc: 76.72%
	 Val. Loss: 0.453 |  Val. Acc: 79.55%
Epoch: 05 | Epoch Time: 0m 24s
	Train Loss: 0.415 | Train Acc: 81.43%
	 Val. Loss: 0.403 |  Val. Acc: 83.60%
Epoch: 06 | Epoch Time: 0m 24s
	Train Loss: 0.349 | Train Acc: 85.02%
	 Val. Loss: 0.337 |  Val. Acc: 86.41%
Epoch: 07 | Epoch Time: 0m 24s
	Train Loss: 0.308 | Train Acc: 86.93%
	 Val. Loss: 0.344 |  Val. Acc: 85.36%
Epoch: 08 | Epoch Time: 0m 24s
	Train Loss: 0.279 | Train Acc: 88.49%
	 Val. Loss: 0.315 |  Val. Acc: 87.62%
Epoch: 09 | Epoch Time: 0m 24s
	Train Loss: 0.252 | Train Acc: 89.98%
	 Val. Loss: 0.326 |  Val. Acc: 88.16%
Epoch: 10 | Epoch T

In [32]:
model.load_state_dict(torch.load('bilstm-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.307 | Test Acc: 88.17%


In [33]:
def predict_sentiment(tokenizer, vocab, model, device, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    length = torch.LongTensor([len(tokens)]).to(device)
    indexes = [vocab.stoi[token] for token in tokens]
    tensor = torch.LongTensor(indexes).unsqueeze(-1).to(device)
    prediction = model(tensor, length)
    probabilities = nn.functional.softmax(prediction, dim = -1)
    pos_probability = probabilities.squeeze()[-1].item()
    return pos_probability

In [34]:
sentence = 'the absolute worst movie of all time.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.015085793100297451

In [35]:
sentence = 'one of the greatest films i have ever seen in my life.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.991886556148529

In [36]:
sentence = "i thought it was going to be one of the greatest films i have ever seen in my life, \
but it was actually the absolute worst movie of all time."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.6712993383407593

In [37]:
sentence = "i thought it was going to be the absolute worst movie of all time, \
but it was actually one of the greatest films i have ever seen in my life."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.28944137692451477